In [9]:
#Matt
#install.packages("RCurl")
library(RCurl)
library(tidyverse, warn.conflicts = FALSE)
library(lubridate, warn.conflicts = FALSE)

#Jensen
library(xml2)
library(rvest)
library(httr)
#install.packages("readxl")
library(readxl)
library(data.table)


Attaching package: ‘rvest’


The following object is masked from ‘package:readr’:

    guess_encoding



Attaching package: ‘data.table’


The following objects are masked from ‘package:lubridate’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose




## Webscrape river flows
Matt

In [10]:
#Matt

row_calc1 <- function(tibble){
    n = as.integer(nrow(tibble))
    data <- tail(tibble, (n-4))
    names(data) <- c('Time','Value(m^3/s)')
    return(data)
}

row_calc2 <- function(tibble){
    n = as.integer(nrow(tibble))
    data <- tail(tibble, (n-4)) %>% select(-X) 
    names(data) <- c('Time','Value(m^3/s)')
    return(data)  
}

opuha <- as_tibble(read.csv(text = getURL("https://hydrowebportal.niwa.co.nz/Export/BulkExport?DateRange=Custom&StartTime=2015-01-01%2000%3A00&EndTime=2020-12-31%2000%3A00&TimeZone=12&Calendar=CALENDARYEAR&Interval=Monthly&Step=1&ExportFormat=csv&TimeAligned=True&RoundData=False&IncludeGradeCodes=False&IncludeApprovalLevels=False&IncludeInterpolationTypes=False&Datasets[0].DatasetName=Discharge%20Value.NRWQN%40TK3&Datasets[0].Calculation=Aggregate&Datasets[0].UnitId=140"))) %>% row_calc2()
"Opuha checkpoint"
rakaia <- as_tibble(read.csv(text = getURL("https://hydrowebportal.niwa.co.nz/Export/BulkExport?DateRange=Custom&StartTime=2015-01-01%2000%3A00&EndTime=2020-12-31%2000%3A00&TimeZone=12&Calendar=CALENDARYEAR&Interval=Monthly&Step=1&ExportFormat=csv&TimeAligned=True&RoundData=False&IncludeGradeCodes=False&IncludeApprovalLevels=False&IncludeInterpolationTypes=False&Datasets[0].DatasetName=Discharge.Master%4068526&Datasets[0].Calculation=Instantaneous&Datasets[0].UnitId=140"))) %>% row_calc2()
"Rakaia checkpoint"
waitaki <- as_tibble(read.csv(text = getURL("https://hydrowebportal.niwa.co.nz/Export/BulkExport?DateRange=Custom&StartTime=2015-01-01%2000%3A00&EndTime=2020-12-31%2000%3A00&TimeZone=12&Calendar=CALENDARYEAR&Interval=PointsAsRecorded&Step=1&ExportFormat=csv&TimeAligned=True&RoundData=False&IncludeGradeCodes=False&IncludeApprovalLevels=False&IncludeInterpolationTypes=False&Datasets[0].DatasetName=Discharge%20Value.NRWQN%40TK6&Datasets[0].Calculation=Instantaneous&Datasets[0].UnitId=140"))) %>% row_calc1()
"Waitaki checkpoint"
kawarau <- as_tibble(read.csv(text = getURL("https://hydrowebportal.niwa.co.nz/Export/BulkExport?DateRange=Custom&StartTime=2015-01-01%2000%3A00&EndTime=2020-12-31%2000%3A00&TimeZone=12&Calendar=CALENDARYEAR&Interval=PointsAsRecorded&Step=1&ExportFormat=csv&TimeAligned=True&RoundData=False&IncludeGradeCodes=False&IncludeApprovalLevels=False&IncludeInterpolationTypes=False&Datasets[0].DatasetName=Discharge%20Value.NRWQN%40AX2&Datasets[0].Calculation=Instantaneous&Datasets[0].UnitId=140"))) %>% row_calc1()                                    
"Kawarau checkpoint"
clutha <- as_tibble(read.csv(text = getURL("https://hydrowebportal.niwa.co.nz/Export/BulkExport?DateRange=Custom&StartTime=2015-01-01%2000%3A00&EndTime=2020-12-31%2000%3A00&TimeZone=12&Calendar=CALENDARYEAR&Interval=Monthly&Step=1&ExportFormat=csv&TimeAligned=True&RoundData=False&IncludeGradeCodes=False&IncludeApprovalLevels=False&IncludeInterpolationTypes=False&Datasets[0].DatasetName=Discharge.Master%4075207&Datasets[0].Calculation=Instantaneous&Datasets[0].UnitId=140"))) %>% row_calc2()                  
"Clutha checkpoint"
mataura <- as_tibble(read.csv(text = getURL("https://hydrowebportal.niwa.co.nz/Export/BulkExport?DateRange=Custom&StartTime=2015-01-01%2000%3A00&EndTime=2020-12-31%2000%3A00&TimeZone=12&Calendar=CALENDARYEAR&Interval=PointsAsRecorded&Step=1&ExportFormat=csv&TimeAligned=True&RoundData=False&IncludeGradeCodes=False&IncludeApprovalLevels=False&IncludeInterpolationTypes=False&Datasets[0].DatasetName=Discharge%20Value.NRWQN%40DN5&Datasets[0].Calculation=Instantaneous&Datasets[0].UnitId=140"))) %>% row_calc1() 
"Mataura checkpoint"
#empty data sets, monowai <- read.csv(text = getURL("https://hydrowebportal.niwa.co.nz/Export/BulkExport?DateRange=Custom&StartTime=2020-01-01%2000%3A00&EndTime=2020-12-31%2000%3A00&TimeZone=12&Calendar=CALENDARYEAR&Interval=PointsAsRecorded&Step=1&ExportFormat=csv&TimeAligned=True&RoundData=False&IncludeGradeCodes=False&IncludeApprovalLevels=False&IncludeInterpolationTypes=False&Datasets[0].DatasetName=Discharge%20Value.NRWQN%40DN10&Datasets[0].Calculation=Instantaneous&Datasets[0].UnitId=140"))                    
#taieri <- read.csv(text = getURL("https://hydrowebportal.niwa.co.nz/Export/BulkExport?DateRange=Custom&StartTime=2020-01-01%2000%3A00&EndTime=2020-12-31%2000%3A00&TimeZone=12&Calendar=CALENDARYEAR&Interval=PointsAsRecorded&Step=1&ExportFormat=csv&TimeAligned=True&RoundData=False&IncludeGradeCodes=False&IncludeApprovalLevels=False&IncludeInterpolationTypes=False&Datasets[0].DatasetName=Discharge%20Value.NRWQN%40DN3&Datasets[0].Calculation=Instantaneous&Datasets[0].UnitId=140"))





[1] "Opuha checkpoint"

[1] "Rakaia checkpoint"

[1] "Waitaki checkpoint"

[1] "Kawarau checkpoint"

[1] "Clutha checkpoint"

[1] "Mataura checkpoint"

## Webscrape hydro stations
Jensen

### Information variables

In [19]:
# Main site address
main_url <- "https://www.emi.ea.govt.nz"
# Sub address for electricity generation information
power_url <- "/Wholesale/Datasets/Generation/Generation_MD"
# Sub address for hydrostations fleet
stations_url <- "/Wholesale/Datasets/Generation/Generation_fleet/Existing"
# Sub address for network supply points
points_url <- "/Wholesale/Reports/R_NSPL_DR?_si=v|3"

### Retrieve generation output data

In [88]:
# Acquire html code from sub address
#source <- paste(main_url, power_url, sep="") %>% read_html()
"Source checkpoint"
# Identify all links in the given html class
all_links <- source %>% html_nodes(".table") %>% html_elements("a") %>% html_attr("href")
"Links identification checkpoint"
# Identify appropriate urls within links and remove duplicates
filtered_links <- all_links[startsWith(all_links, power_url) & !duplicated(all_links)]
"Links filtered checkpoint"
# Extract date information from filtered links
start_pos <- nchar(power_url) + 2
dates <- substring(filtered_links , start_pos, start_pos + 3) 
"Date extraction checkpoint"
# Set dates wanted
dates_wanted <- seq(from=2015,to=2020)
# Select links containing csv data
selected_links <- filtered_links[dates %in% dates_wanted]
"Links selected checkpoint"
# Extract csv data into dataframes
link <- paste(main_url, selected_links, sep="") 
generation_md <- as.list(link) %>% lapply(read_csv)
"Generation retrieval checkpoint"

[1] "Source checkpoint"

[1] "Links identification checkpoint"

[1] "Links filtered checkpoints"

[1] "Date extraction checkpoint"

[1] "Links selected checkpoint"

...

Rows: 2170 Columns: 57

── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr   (6): Site_Code, POC_Code, Nwk_Code, Gen_Code, Fuel_Code, Tech_Code
dbl  (48): TP1, TP2, TP3, TP4, TP5, TP6, TP7, TP8, TP9, TP10, TP11, TP12, TP...
lgl   (2): TP49, TP50
date  (1): Trading_date


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.

Rows: 2170 Columns: 57

── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr   (6): Site_Code, POC_Code, Nwk_Code, Gen_Code, Fuel_Code, Tech_Code
dbl  (48): TP1, TP2, TP3, TP4, TP5, TP6, TP7, TP8, TP9, TP10, TP11, TP12, TP...
lgl   (2): TP49, TP50
date  (1): Trading_date


ℹ Use `spec()` to retri

Rows: 2139 Columns: 57

── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr   (6): Site_Code, POC_Code, Nwk_Code, Gen_Code, Fuel_Code, Tech_Code
dbl  (48): TP1, TP2, TP3, TP4, TP5, TP6, TP7, TP8, TP9, TP10, TP11, TP12, TP...
lgl   (2): TP49, TP50
date  (1): Trading_date


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.

Rows: 2070 Columns: 57

── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr   (6): Site_Code, POC_Code, Nwk_Code, Gen_Code, Fuel_Code, Tech_Code
dbl  (48): TP1, TP2, TP3, TP4, TP5, TP6, TP7, TP8, TP9, TP10, TP11, TP12, TP...
lgl   (2): TP49, TP50
date  (1): Trading_date


ℹ Use `spec()` to retri

Rows: 2170 Columns: 57

── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr   (6): Site_Code, POC_Code, Nwk_Code, Gen_Code, Fuel_Code, Tech_Code
dbl  (48): TP1, TP2, TP3, TP4, TP5, TP6, TP7, TP8, TP9, TP10, TP11, TP12, TP...
lgl   (2): TP49, TP50
date  (1): Trading_date


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.

Rows: 2088 Columns: 57

── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr   (6): Site_Code, POC_Code, Nwk_Code, Gen_Code, Fuel_Code, Tech_Code
dbl  (48): TP1, TP2, TP3, TP4, TP5, TP6, TP7, TP8, TP9, TP10, TP11, TP12, TP...
lgl   (2): TP49, TP50
date  (1): Trading_date


ℹ Use `spec()` to retri

Rows: 1954 Columns: 57

── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr   (6): Site_Code, POC_Code, Nwk_Code, Gen_Code, Fuel_Code, Tech_Code
dbl  (48): TP1, TP2, TP3, TP4, TP5, TP6, TP7, TP8, TP9, TP10, TP11, TP12, TP...
lgl   (2): TP49, TP50
date  (1): Trading_date


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.

Rows: 2170 Columns: 57

── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr   (6): Site_Code, POC_Code, Nwk_Code, Gen_Code, Fuel_Code, Tech_Code
dbl  (48): TP1, TP2, TP3, TP4, TP5, TP6, TP7, TP8, TP9, TP10, TP11, TP12, TP...
lgl   (2): TP49, TP50
date  (1): Trading_date


ℹ Use `spec()` to retri

[1] "Generation retrieval checkpoint"

In [89]:
length(generation_md)

[1] 72

In [86]:
col1 <- c(1,1,2,2,3,4,5)
col2 <- c('a','b','c','d','e','f','g')
test <- tibble(col1,col2) 
val <- seq(from=1,to=2)
alist <- c(1,1,2,2,5,6,7)
test[alist %in% val,]

col1,col2
<dbl>,<chr>
1,a
1,b
2,c
2,d


In [ ]:
#Sujung

In [ ]:
#Graphing code